### Convolutional Neural Network
<b>Key Layers in a CNN: </b> <br>

<small>

1. <b>Convolutional Layer:</b> Applies a filter to the input image to produce feature maps, capturing spatial relationships in the image

    - Feature Extraction: Convolution helps the network find important features like edges, shapes, or textures in the image, without needing to tell it exactly what to look for.

    - Local Connectivity: Instead of looking at the entire image at once, the filter focuses on small parts (local patterns), which makes it more efficient and able to detect detailed features.

        <br>

- <b>ReLU Activation:</b> Is applied element-wise to the feature map produced by the convolutional layer. Introduces non-linearity by setting all negative values to zero, allowing the network to learn complex patterns.

    <br>

Together, they form the basic building block in CNNs, where the <b>convolutional layer extracts features</b>, and the <b>ReLU activation enables non-linear decision boundaries</b>.

<br>


2. <b>Pooling Layer:</b> Reduces the spatial dimensions of the feature maps, retaining essential information while reducing computation, prevents overfitting, and translating invariance
    - Max/ Min/ Average/ Sum pooling: take the max/ min/ average/ sum value in a region

        <br>


3. <b>Flattening Layer:</b> Converts the 2D (or higher-dimensional) feature maps into a 1D array (vector) so that they can be fed into the fully connected layer

    <br>


4. <b>Fully Connected Layer: </b> Combines the features from the previous layers to make final predictions

    <br>

<b>Finally, in Output</b> layer <b>Softmax Cross-Entropy</b> is just a fancy way to:

    1. Turn the raw scores (logits) into probabilities. Then pick the highest one to make final decision (softmax)

    2. Check how wrong the prediction is using penalties (cross-entropy)
    
    3. Help the box learn and imporve!🎯


</small>

Importing the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.18.0'

#### Part 1 - Data Preprocessing

Preprocessing the Training set
- some transformation to avoid overfitting

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,        # feature scaling --> Normalization (0, 1)
    shear_range=0.2,       # Apply shear transformations
    zoom_range=0.2,        # Random zoom
    horizontal_flip=True  # Randomly flip images horizontally
)

In [4]:
training_set = train_datagen.flow_from_directory(
    'dataset/training_set/',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 8000 images belonging to 2 classes.


Preprocessing the Test set

In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(
    'dataset/test_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.


#### Part 2 - Building the CNN

Initializing the CNN

In [6]:
cnn = tf.keras.models.Sequential()

Step 1 - Convolution <br>

<small>

- filters=32 --> the number of filters (or kernels) the convolutional layer will use

- kernel_size=3 --> the filter is a 3x3 matrix

- input_shape=[64, 64, 3] --> height and width 64 pixels, and 3 channels (RGB, for color images)

<br>


You can be confident that 32 filters cover the whole image because:

    1. Every filter scans the entire image systematically

    2. Filters are applied one by one to all parts of the image

    3. They work together, producing 32 feature maps that represent different aspects of the image

So, the whole image is always covered—what changes is what features are learned by the filters.

</small>

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Step 2 - Pooling

In [8]:
# pool_size=n --> n X n
# strides=n --> step size n and skip overlapping
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

Adding a second convolutional layer

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

Step 3 - Flattening

In [10]:
cnn.add(tf.keras.layers.Flatten())

Step 4 - Full Connection

In [11]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

Step 5 - Output Layer

In [12]:
# binary classification
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

#### Part 3 - Training the CNN

Compiling the CNN

In [13]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Training the CNN on the Training set and evaluating it on the Test set <br>

In [24]:
cnn.fit(x=training_set, validation_data=test_set, epochs=15) # epochs = 25

Epoch 1/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 139s 557ms/step - accuracy: 0.9069 - loss: 0.2404 - val_accuracy: 0.8030 - val_loss: 0.5345
Epoch 2/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 26s 104ms/step - accuracy: 0.9063 - loss: 0.2247 - val_accuracy: 0.7880 - val_loss: 0.6029
Epoch 3/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 25s 98ms/step - accuracy: 0.9180 - loss: 0.2017 - val_accuracy: 0.7890 - val_loss: 0.5740
Epoch 4/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 24s 96ms/step - accuracy: 0.9213 - loss: 0.1986 - val_accuracy: 0.7875 - val_loss: 0.6194
Epoch 5/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 23s 93ms/step - accuracy: 0.9251 - loss: 0.1867 - val_accuracy: 0.7810 - val_loss: 0.6034
Epoch 6/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 24s 95ms/step - accuracy: 0.9301 - loss: 0.1856 - val_accuracy: 0.7960 - val_loss: 0.6066
Epoch 7/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 25s 99ms/step - accuracy: 0.9229 - loss: 0.1798 - val_accuracy: 0.7940 - val_loss: 0.6427
Epoch 8/15
250/250 ━━━━━━━━━━━━━━━━━━━━ 24s 96ms/step - accuracy: 0.9347 - loss: 0.1663

#### Part 4 - Making a single prediction

In [36]:
import numpy as np
from tensorflow.keras.preprocessing import image

# load_img --> channels=3
test_image = image.load_img('dataset/cat.png', target_size=(64, 64)) # after this step --> (64, 64, 3)

test_image = image.img_to_array(test_image)

# (batch_size, height, width, channels)
# expand_dims --> batch_size=1
test_image = np.expand_dims(test_image, axis=0) # axis=0 --> at the beginning

test_image = test_image / 255.0

result = cnn.predict(test_image)

training_set.class_indices
print(training_set.class_indices)

if result[0][0] > 0.5:
    prediction = 'dog'
else:
    prediction = 'cat'
    

# if round(result[0][0]) == 1:
#     prediction = 'dog'
# else:
#     prediction = 'cat'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
{'cats': 0, 'dogs': 1}


In [23]:
print(prediction)

cat
